In [9]:
import pandas as pd
import sklearn
sklearn.__version__
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import pickle

In [3]:
def read_dataset(filename):
    df=pd.read_parquet(filename)
    df.tpep_dropoff_datetime=pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime=pd.to_datetime(df.tpep_pickup_datetime)
    df['duration']=df.tpep_dropoff_datetime-df.tpep_pickup_datetime
    df['duration'] = df['duration'].dt.total_seconds().div(60).astype(float)
    #df=df[(df.duration>=1)&(df.duration<=60)]
    categorical=['PULocationID','DOLocationID']
    numerical=['trip_distance']
    df[categorical]=df[categorical].astype(str)
    return df
    

In [4]:
df_train=read_dataset('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
df_test=read_dataset(filename='https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')


In [5]:
len(df_train),len(df_test)

(2964624, 3007526)

In [6]:
categorical=['PULocationID','DOLocationID']
numerical=['trip_distance']
target='duration'
dv=DictVectorizer()
train_dicts=df_train[categorical+numerical].to_dict(orient='records')
test_dicts=df_test[categorical+numerical].to_dict(orient='records')
X_train=dv.fit_transform(train_dicts)
X_test=dv.transform(test_dicts)



In [7]:
target='duration'
y_train=df_train[target].values
y_test=df_test[target].values


In [10]:
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
mean_squared_error(y_test,y_pred,squared=False)

33.49205886470693

In [12]:
with open ('models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv,lr),f_out)